### 전이학습 

학습된 모델이 있다고 하면 특성추출기 빼고 다른 네트워크를 학습

장점 1. 학습시간 줄고  
장점 2. 데이터 부족한 상황에서도 가능.

### 참고

https://github.com/hanbitmedia/PyTorch-Hanbit/tree/master/Week6_TransferLearning/1_StyleTransfer

여기 이번 문제랑 비슷해가지고요. 공부해봤습니다. 

In [1]:
!pip install torchinfo

In [2]:
import torch
import torch.nn as nn

from torchinfo import summary

from torchvision import models
resnet=models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

![](https://cdn-5f733ed3c1ac190fbc56ef88.closte.com/wp-content/uploads/2019/07/ResNet50_architecture-1.png)

In [5]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0=nn.Sequential(*list(resnet.children())[:-1]) # fc_layer 필요없고
        self.layer1=nn.Sequential(
        nn.Linear(2048,500), # 2048로 떨어지는 avgpool에 이어 2048개의 features 와 500개 features를 이어주는 custom fc_layer
        nn.BatchNorm1d(500),
        nn.ReLU(),
        nn.Linear(500,26), # 한번더 
        nn.ReLU())
        
    def forward(self,x):
        out=self.layer0(x)
        out=out.view(out.size(0),-1) # flatten
        out=self.layer1(out)
        return out

model=ResNet()

# 기본 layer0는 공부하지 않는다.
for params in model.layer0.parameters():
    params.require_grad=False
    
for params in model.layer1.parameters():
    params.require_grad=True

In [6]:
summary(model,(1,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [1, 2048, 1, 1]           --
|    └─Conv2d: 2-1                       [1, 64, 112, 112]         9,408
|    └─BatchNorm2d: 2-2                  [1, 64, 112, 112]         128
|    └─ReLU: 2-3                         [1, 64, 112, 112]         --
|    └─MaxPool2d: 2-4                    [1, 64, 56, 56]           --
|    └─Sequential: 2-5                   [1, 256, 56, 56]          --
|    |    └─Bottleneck: 3-1              [1, 256, 56, 56]          75,008
|    |    └─Bottleneck: 3-2              [1, 256, 56, 56]          70,400
|    |    └─Bottleneck: 3-3              [1, 256, 56, 56]          70,400
|    └─Sequential: 2-6                   [1, 512, 28, 28]          --
|    |    └─Bottleneck: 3-4              [1, 512, 28, 28]          379,392
|    |    └─Bottleneck: 3-5              [1, 512, 28, 28]          280,064
|    |    └─Bottleneck: 3-6              [1, 512, 28, 28]  